<a href="https://colab.research.google.com/github/deepchatterjeevns/Pytorch-Udacity-Challenge/blob/master/intro-to-pytorch/Part%208%20-%20Transfer%20Learning%20(Exercises%20Solved).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
try:
       import torch
except:
    import os
    os.environ['TCMALLOC_LARGE_ALLOC_REPORT_THRESHOLD']='2000000000'
    # http://pytorch.org/
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
    
    !{sys.executable} -m pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision >/dev/null

import torch
from IPython.core.display import Image

In [0]:
# get helper functions

! wget  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/fc_model.py >/dev/null 2>&1
! wget  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py  >/dev/null 2>&1

In [3]:
!ls

Cat_Dog_data	  fc_model.py	 helper.py    __MACOSX
Cat_Dog_data.zip  fc_model.py.1  helper.py.1  sample_data


In [6]:
!wget -c https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip

# remove existing directories
! rm -r Cat_Dog_data __MACOSX || true
!unzip -qq Cat_Dog_data.zip

--2018-12-17 05:27:05--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.21.157
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.21.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580495262 (554M) [application/zip]
Saving to: ‘Cat_Dog_data.zip’

Cat_Dog_data.zip    100%[===================>] 553.60M  1.14MB/s    in 85s     

2018-12-17 05:28:31 (6.51 MB/s) - ‘Cat_Dog_data.zip’ saved [580495262/580495262]

rm: cannot remove 'Cat_Dog_data': No such file or directory
rm: cannot remove '__MACOSX': No such file or directory


In [20]:
!ls

Cat_Dog_data	  fc_model.py	 helper.py    __MACOSX
Cat_Dog_data.zip  fc_model.py.1  helper.py.1  sample_data


In [14]:
import PIL
print(PIL.PILLOW_VERSION)

4.0.0


In [15]:
!pip install -U pillow

Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (5.3.0)


# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [0]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [16]:
model = models.densenet121(pretrained=True)
model

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [0]:
import time

In [9]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 7.202 seconds
Device = cuda; Time per batch: 0.019 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

## Solution Given

In [18]:
## TODO: Use a pretrained model to classify the cat and dog images


# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device);

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [19]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.888.. Test loss: 0.409.. Test accuracy: 0.786
Epoch 1/1.. Train loss: 0.544.. Test loss: 0.314.. Test accuracy: 0.864
Epoch 1/1.. Train loss: 0.339.. Test loss: 0.149.. Test accuracy: 0.958
Epoch 1/1.. Train loss: 0.231.. Test loss: 0.106.. Test accuracy: 0.967
Epoch 1/1.. Train loss: 0.198.. Test loss: 0.084.. Test accuracy: 0.971
Epoch 1/1.. Train loss: 0.162.. Test loss: 0.077.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.168.. Test loss: 0.059.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.199.. Test loss: 0.064.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.151.. Test loss: 0.053.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.155.. Test loss: 0.071.. Test accuracy: 0.973
Epoch 1/1.. Train loss: 0.201.. Test loss: 0.053.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.193.. Test loss: 0.055.. Test accuracy: 0.982
Epoch 1/1.. Train loss: 0.215.. Test loss: 0.061.. Test accuracy: 0.976
Epoch 1/1.. Train loss: 0.242.. Test loss: 0.057.. Test accuracy

### My solution Using Resnet50

In [21]:
# importing and loading new model
model = models.resnet50(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [23]:

# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict

#Define new classifier
classifier = nn.Sequential(nn.Linear(2048,512),
                          nn.ReLU(),
                          nn.Dropout(p=.2),
                          nn.Linear(512,2),
                          nn.LogSoftmax(dim=1))

#Attach classifier to model
model.fc = classifier

#Define Loss and Criterion
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr = .003)

model.to(device)




ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
#Assign Variables for training
epochs = 1
steps = 0
running_loss = 0
print_every = 5


In [24]:
#Loop through Epochs:
for epoch in range(epochs):
    
    #Train loop
    for image, label in trainloader:
        image, label = image.cuda(), label.cuda()
        steps += 1
        
        
        
        optimizer.zero_grad()
        log_ps = model(image)
        loss = criterion(log_ps, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        #Print every so often to test output .. set model eval to turn of grads dropouts etc
        if steps % print_every == 0:
            model.eval()
            test_loss = 0
            accuracy = 0
            
            #Test loop
            for image, label in testloader:
                image, label = image.cuda(), label.cuda()
                log_ps = model(image)
                loss = criterion(log_ps, label)
                test_loss += loss.item()
                
                #Calculate our accuracy
                ps = torch.exp(log_ps)
                top_ps, top_class= ps.topk(1, dim = 1)
                equality = top_class == label.view(*top_class.shape)
                accuracy += torch.mean(equality.type(torch.FloatTensor))
            
            #for running loss its print_every bc only parts of trainloader are running while all of test loader
            print("Epoch: {}/{}..".format(epoch+1,epochs),
                  "Accuracy: {:.3f}%".format(accuracy/len(testloader)),
                  "test Loss: {:.3f}..".format(test_loss/len(testloader)),
                  "training loss: {:3f}..".format(running_loss/print_every))
            
            running_loss = 0
            model.train()

Epoch: 1/1.. Accuracy: 0.968% test Loss: 0.125.. training loss: 1.583493..
Epoch: 1/1.. Accuracy: 0.972% test Loss: 0.081.. training loss: 0.329148..
Epoch: 1/1.. Accuracy: 0.975% test Loss: 0.065.. training loss: 0.238597..
Epoch: 1/1.. Accuracy: 0.973% test Loss: 0.072.. training loss: 0.162572..
Epoch: 1/1.. Accuracy: 0.973% test Loss: 0.067.. training loss: 0.180534..
Epoch: 1/1.. Accuracy: 0.979% test Loss: 0.055.. training loss: 0.181060..
Epoch: 1/1.. Accuracy: 0.977% test Loss: 0.055.. training loss: 0.168205..
Epoch: 1/1.. Accuracy: 0.980% test Loss: 0.054.. training loss: 0.219233..
Epoch: 1/1.. Accuracy: 0.979% test Loss: 0.054.. training loss: 0.149367..
Epoch: 1/1.. Accuracy: 0.970% test Loss: 0.077.. training loss: 0.128115..
Epoch: 1/1.. Accuracy: 0.962% test Loss: 0.092.. training loss: 0.170408..
Epoch: 1/1.. Accuracy: 0.974% test Loss: 0.067.. training loss: 0.137803..
Epoch: 1/1.. Accuracy: 0.970% test Loss: 0.074.. training loss: 0.177233..
Epoch: 1/1.. Accuracy: 0.